In [27]:
import time
import random
import pygame
import numpy as np
import subprocess
from screeninfo import get_monitors

def calc_speed(spd, dfs, fps, disp):
    speed = np.array(spd) * np.pi / 180 #numpy arrray of speed in visual angle
    tsp = np.tan(speed)
    ss = np.array(mon_size()[disp][0]) #screen size in mm
    sr = np.array(mon_size()[disp][1])
    sw_r = round(sr[0]/ss[0], 2)
    sh_r = round(sr[1]/ss[1], 2)
        
    sx = int(tsp[0] * dfs * sw_r / fps)
    sy = int(tsp[1] * dfs * sh_r / fps)
    asx = np.arctan(sx * fps / (sw_r * dfs)) * 180 / np.pi
    asy = np.arctan(sy * fps / (sh_r * dfs)) * 180 / np.pi

    s_ball = [sx, sy]
    av_ball = [asx, asy]
    
    return s_ball, av_ball

def mon_size():
    dml = []
    for m in get_monitors():
        wl = m.width_mm; hl = m.height_mm;
        wp = m.width; hp = m.height;
        dml.append([[round(wl,1), round(hl,1)], [wp, hp]])
    return dml

def calibration_points(rb, disp):
    disp_size = mon_size()[disp]
    ssize = np.array(mon_size()[disp][0]) 
    srez = np.array(mon_size()[disp][1])
    diam = rb*2

    binelh, binerh = [diam, srez[0]-diam]
    bineuv, binedv = [diam, srez[1]-diam]

    midh = math.ceil(srez[0]/2)
    midv = math.ceil(srez[1]/2)

    midlh = math.ceil((midh-diam)/2 + diam)
    midrh = math.ceil((binerh - midh)/2 + midh)
    # midlv = math.ceil((midh-rb)/2 + rb)
    # midrv = math.ceil((bin_edge_h - midh)/2)

    calib_pts = []

    calib_pts.append([binelh, bineuv])
    calib_pts.append([midlh, bineuv])
    calib_pts.append([midh, bineuv])
    calib_pts.append([midrh, bineuv])
    calib_pts.append([binerh, bineuv])

    calib_pts.append([binelh, midv])
    calib_pts.append([midlh, midv])
    calib_pts.append([midh, midv])
    calib_pts.append([midrh, midv])
    calib_pts.append([binerh, midv])

    calib_pts.append([binelh, binedv])
    calib_pts.append([midlh, binedv])
    calib_pts.append([midh, binedv])
    calib_pts.append([midrh, binedv])
    calib_pts.append([binerh, binedv])
    
    dens = ssize/srez
    bin_size = [(binerh-binelh)/4, (binedv - bineuv)/2]
    
    return calib_pts, bin_size


# initialize pygame, sound player
pygame.init()
pygame.mixer.init()

deets = pygame.display.Info() #Collects the resolution of your screen 

#----------------EDIT HERE ONLY---------------------------------------------------------------------------------------------------------------------------------

vol = 0.1          #set volume
ntrial = 1         #number of trials
rb = 40            #radius of the ball
app_dur = [3, 7]   #time for which ball is seen
fl_dur = 0.15      #Flicker duration
click_window = 1   #Time after flicker within which the subject must respond 
set_fps = 60       #Set fps of game
rend_stats = True  #render fps and time elapsed 
vas = [20, 10]     #set speed in visual angle/sec 
dfs = 800          #Determine approximte distance from monitor in mm
disp = 0           #Set output display
audio = True       #Whether you want to play audio or not

# define colors 
col_bb = (255, 0, 0)                   #colour of ball when bouncing
col_flk = (200, 0, 50)                 #colour of ball during flicker
col_bg = (0, 0, 0)                     #colour of background
fixation_color = (255, 255, 255)       #colour of ball at the start of trial - fixation point

#----------------------------------------------------------------------------------------------------------------------------------------------------------------

trial_order, size_bin = calibration_points(rb, disp)
trial_order = np.array(trial_order)
np.random.shuffle(trial_order)
# Calculates the pixel speed based on your choice of visual angle speed. 
# NOTE - you need to specify an approximate distance from screen. 
spd, avas = calc_speed(vas, dfs, set_fps, disp) 


# defining width and height of screen - Collected from the info taken of the screen.
infoObject = pygame.display.Info()
width = infoObject.current_w
height = infoObject.current_h
screen_res = (width, height)

if audio:
    right_ans = pygame.mixer.Sound("/home/yramakrishna/DeepLabCut/conda-environments/Sounds/reward.wav")
    wrong_ans = pygame.mixer.Sound("/home/yramakrishna/DeepLabCut/conda-environments/Sounds/failure.wav")
    right_ans.set_volume(vol)
    wrong_ans.set_volume(vol)

clock = pygame.time.Clock()

pygame.display.set_caption("GFG Bouncing game")
screen = pygame.display.set_mode((screen_res), pygame.FULLSCREEN, display=disp) #renders it to the entire screen

# screen = pygame.display.set_mode((0,0),pygame.FULLSCREEN)

correct_clicks = 0
wrong_clicks = 0
tooerc = 0
trial = 1

llb, lub = app_dur
# timec = clock.get_time()
timec = pygame.time.get_ticks()


while len(trial_order)>0:
    pygame.event.clear()
    rspt = random.randint(0, len(trial_order)-1)
    # Define the fixation point coordinates
    fixation = trial_order[rspt]
    print(fixation)
    fixation_x, fixation_y = fixation #randomize this
    fixation_radius = rb

    # Draw the fixation point
    pygame.draw.circle(screen, fixation_color, (fixation_x, fixation_y), fixation_radius)

    # Update the display
    pygame.display.flip()

    # Wait for a mouse click on the fixation point to start the trial
    trial_started = False
    while not trial_started:
        for event in pygame.event.get():
            if event.type == pygame.KEYDOWN and event.key == pygame.K_SPACE:
                trial_started = True

    # define ball
    ball_obj = pygame.draw.circle(
        surface=screen, color=col_bb, center=fixation, radius=rb)
    
    start_pt = fixation
    
    # define speed of ball
    
    # speed = [X direction speed, Y direction speed] in pixels per frame
    
    speed = spd

    limit = random.randint(llb, lub) 

    last_seen = []
    all_points = []

    start_time = time.time()
    flick_em1 = random.randint(0,1)
    flick_em = bool(flick_em1)
    flickr = limit-(fl_dur+click_window)
    bound = [start_time + flickr, start_time+flickr+fl_dur]
    
    stptx = min(start_pt[0], width-100)
    stpty = min(start_pt[1], height-100)
    
    random.seed(time.time())
    iwr = width #random.randint(stptx, width)
    iwl = 0 #random.randint(stptx, 0)
    iwd = height #random.randint(stpty, height)
    iwu = 0 #random.randint(stpty, 0)
    click = False #registering clicks
    tooer = False #registering early clicks
    
    # game loop
    while True:
        # event loop

        if (time.time() - start_time) >= limit:
            last_seen = ball_obj.center
            break

        # fill background colour on screen color on screen
        screen.fill(col_bg)

        # move the ball
        # Let center of the ball is (100,100) and the speed is (1,1)
        ball_obj1 = ball_obj.move(speed)
        # Now center of the ball is (101,101)
        # In this way our wall will move

        # if ball goes out of screen then change direction of movement --------------------------------------------------------------------------------------------------------
        if ball_obj.left <= 0 or ball_obj.right >= width:
            speed[0] = -speed[0]
        if ball_obj.top <= 0 or ball_obj.bottom >= height:
            speed[1] = -speed[1]
        
        # #ball changes movement randomly -------------------------------------------------------------------------------------------------------------------------------------
        # if ball_obj1.left <= max(iwl, 0): 
        #     speed[0] = -speed[0]
        #     iwr = random.randint(min(ball_obj.right + 300, width), width)
        # if ball_obj1.right >= min(iwr, width):
        #     speed[0] = -speed[0]
        #     iwl = random.randint(0, max(ball_obj.left - 300, 0))
        # if ball_obj1.top <= max(iwu, 0):
        #     speed[1] = -speed[1]
        #     iwd = random.randint(min(ball_obj.bottom + 300, height), height)
        # if ball_obj1.bottom >= min(height,iwd):
        #     speed[1] = -speed[1]
        #     iwu = random.randint(0, max(ball_obj.top - 300, 0))

        ball_obj = ball_obj.move(speed)
        all_points.append(ball_obj.center)

        if (time.time() >= bound[0] and time.time() <= bound[1]) and flick_em:
            pygame.draw.circle(surface=screen, color=col_flk, center=ball_obj.center, radius=rb)  
        # draw ball at new centers that are obtained after moving ball_obj
        else:
            pygame.draw.circle(surface=screen, color=col_bb, center=ball_obj.center, radius=rb)
        
        
        timec = pygame.time.get_ticks()/1000
        clock.tick(set_fps)
        fps_disp = clock.get_fps()

        if rend_stats == True:
            font = pygame.font.SysFont('Arial', 20)
            text4 = font.render(f'Time taken: {timec}', True, (255, 255, 0))
            text5 = font.render(f'FPS: {fps_disp}', True, (255, 255, 0))
            screen.blit(text4, (0, 0))
            screen.blit(text5, (250, 0))

        # update screen
        pygame.display.flip()
        
        if time.time() <= (bound[0]):
            for event in pygame.event.get():
                    if event.type == pygame.KEYDOWN:
                        if event.key == pygame.K_SPACE:
                            tooer = True
            
        else:
            if click != True and (time.time() - start_time) <= limit:
                for event in pygame.event.get():
                    if event.type == pygame.KEYDOWN:
                        if event.key == pygame.K_SPACE:
                            click = True
        if tooer == True:
            break
    
    if tooer == True:
        tooerc += 1
        if audio:
            pygame.mixer.Sound.play(wrong_ans)
    else:
        trial = trial+1
        if click == flick_em:
            correct_clicks += 1
            np.delete(trial_order, rspt, 0)
            if audio:
                pygame.mixer.Sound.play(right_ans)
        else:
            wrong_clicks += 1
            if audio:
                pygame.mixer.Sound.play(wrong_ans)
    
    np.random.shuffle(x)            
    
    pygame.draw.circle(surface=screen, color=col_bg, center=ball_obj.center, radius=rb)
    pygame.display.flip()
    
    pygame.time.wait(1000)

    print(flick_em, click)
    print("")
                    
for i in range(1, 1000): #displays results for a few seconds
    font = pygame.font.SysFont(None, 48)
    text = font.render(f"Correct guess = {correct_clicks}", True, (255, 255, 255))
    text2 = font.render(f"Wrong guess = {wrong_clicks}", True, (255, 255, 255))
    text3 = font.render(f" Too hasty = {tooerc}", True, (255, 255, 255))
    screen.blit(text, (width/2 - text.get_width()/2, height/2 - 3 * text.get_height()/2))
    screen.blit(text2, (width/2 - text2.get_width()/2, height/2))
    screen.blit(text3, (width/2 - text3.get_width()/2, height/2 + 3 * text3.get_height()/2))
    pygame.display.flip()

pygame.display.quit()
exit()

[1400   80]
False False

[960 540]
True True

[1400   80]
True True

[520  80]
False True

[520  80]
True True

[520 540]
True True

[520  80]
False False

[1840 1000]
False False

[80 80]
True False

[960  80]
True True

[960 540]
False False

[1840   80]
True False

[1840   80]
False False

[ 960 1000]
False False

[80 80]
False False

[1840  540]
False False

[1840 1000]
False False

[  80 1000]
True True



IndexError: index 15 is out of bounds for axis 0 with size 15

In [18]:
from datetime import datetime
now = datetime.now();today = now.strftime("%d-%m-%Y_%H-%M-%S")
fl_name = 'Trajectory_' + today + '.txt.'
# d1 = today.strftime("%d-%m-%Y")
print(fl_name)

Trajectory_30-03-2023_10-32-52.txt.


In [2]:
import time
import random
import pygame
import numpy as np
import subprocess
from screeninfo import get_monitors
import math

In [3]:
def calc_speed(spd, dfs, fps, disp):
    speed = np.array(spd) * np.pi / 180 #numpy arrray of speed in visual angle
    tsp = np.tan(speed)
    ss = np.array(mon_size()[disp][0]) #screen size in cms
    sr = np.array(mon_size()[disp][1])
    # print(dml.shape)
    sw_r = round(sr[0]/ss[0], 2)
    sh_r = round(sr[1]/ss[1], 2)
    
    print(speed, sw_r, sh_r)
    
    sx = int(tsp[0] * dfs * sw_r / fps)
    sy = int(tsp[1] * dfs * sh_r / fps)
    asx = np.arctan(sx * fps / (sw_r * dfs)) * 180 / np.pi
    asy = np.arctan(sy * fps / (sh_r * dfs)) * 180 / np.pi

    s_ball = [sx, sy]
    av_ball = [asx, asy]
    
    return s_ball, av_ball

def mon_size():
    dml = []
    for m in get_monitors():
        wl = m.width_mm; hl = m.height_mm;
        wp = m.width; hp = m.height;
        dml.append([[round(wl,1), round(hl,1)], [wp, hp]])
    return dml


In [4]:
vol = 0.1          #set volume
ntrial = 1         #number of trials
rb = 40            #radius of the ball
app_dur = [3, 7]   #time for which ball is seen
fl_dur = 0.15      #Flicker duration
click_window = 1   #Time after flicker within which the subject must respond 
set_fps = 60       #Set fps of game
rend_stats = True  #render fps and time elapsed 
vas = [20, 10]     #set speed in visual angle/sec 
dfs = 800          #Determine approximte distance from monitor in mm
disp = 0           #Set output display
audio = True       #Whether you want to play audio or not

# define colors 
col_bb = (255, 0, 0)                   #colour of ball when bouncing
col_flk = (200, 0, 50)                 #colour of ball during flicker
col_bg = (0, 0, 0)                     #colour of background
fixation_color = (255, 255, 255)       #colour of ball at the start of trial - fixation point

In [33]:
def calibration_points(rb, disp):
    disp_size = mon_size()[disp]
    ssize = np.array(mon_size()[disp][0]) 
    srez = np.array(mon_size()[disp][1])
    diam = rb*2

    binelh, binerh = [diam, srez[0]-diam]
    bineuv, binedv = [diam, srez[1]-diam]

    midh = math.ceil(srez[0]/2)
    midv = math.ceil(srez[1]/2)

    midlh = math.ceil((midh-diam)/2 + diam)
    midrh = math.ceil((binerh - midh)/2 + midh)
    # midlv = math.ceil((midh-rb)/2 + rb)
    # midrv = math.ceil((bin_edge_h - midh)/2)

    calib_pts = []

    calib_pts.append([binelh, bineuv])
    calib_pts.append([midlh, bineuv])
    calib_pts.append([midh, bineuv])
    calib_pts.append([midrh, bineuv])
    calib_pts.append([binerh, bineuv])

    calib_pts.append([binelh, midv])
    calib_pts.append([midlh, midv])
    calib_pts.append([midh, midv])
    calib_pts.append([midrh, midv])
    calib_pts.append([binerh, midv])

    calib_pts.append([binelh, binedv])
    calib_pts.append([midlh, binedv])
    calib_pts.append([midh, binedv])
    calib_pts.append([midrh, binedv])
    calib_pts.append([binerh, binedv])
    
    dens = ssize/srez
    bin_size = [(binerh-binelh)/4, (binedv - bineuv)/2]
    
    return calib_pts, bin_size

x = np.array(calibration_points(rb, disp)[0])
# trial_order = np.random.shuffle(x)
rspt = random.randint(0, len(trial_order))
# Define the fixation point coordinates
fixation = trial_order[rspt]
fixation_x, fixation_y = fixation
print(trial_order.shape)
trial_order = np.delete(trial_order, rspt, 0)
print(trial_order.shape)

(15, 2)
(14, 2)


In [25]:
#BINNING

disp_size = mon_size()[disp]
ssize = np.array(mon_size()[disp][0]) 
srez = np.array(mon_size()[disp][1])
diam = rb*2

binelh, binerh = [diam, srez[0]-diam]
bineuv, binedv = [diam, srez[1]-diam]

midh = math.ceil(srez[0]/2)
midv = math.ceil(srez[1]/2)

midlh = math.ceil((midh-diam)/2 + diam)
midrh = math.ceil((binerh - midh)/2 + midh)
# midlv = math.ceil((midh-rb)/2 + rb)
# midrv = math.ceil((bin_edge_h - midh)/2)

calib_pts = []

calib_pts.append([binelh, bineuv])
calib_pts.append([midlh, bineuv])
calib_pts.append([midh, bineuv])
calib_pts.append([midrh, bineuv])
calib_pts.append([binerh, bineuv])

calib_pts.append([binelh, midv])
calib_pts.append([midlh, midv])
calib_pts.append([midh, midv])
calib_pts.append([midrh, midv])
calib_pts.append([binerh, midv])

calib_pts.append([binelh, binedv])
calib_pts.append([midlh, binedv])
calib_pts.append([midh, binedv])
calib_pts.append([midrh, binedv])
calib_pts.append([binerh, binedv])

dens = ssize/srez
bin_size = [(binerh-binelh)/4, (binedv - bineuv)/2]

print(bin_size)

# calib_pts.append([binelh, binedv])
# calib_pts.append([binerh, binedv])

print(calib_pts)

print(binelh, binerh, bineuv, binedv)
# print(midlh, midrh)
print(midv)

[0.274 0.274]
[440.0, 460.0]
[[80, 80], [520, 80], [960, 80], [1400, 80], [1840, 80], [80, 540], [520, 540], [960, 540], [1400, 540], [1840, 540], [80, 1000], [520, 1000], [960, 1000], [1400, 1000], [1840, 1000]]
80 1840 80 1000
540


In [22]:
dens = ssize/srez
print(np.around(dens, 3))

[0.274 0.274]


In [10]:
# spd, avas = calc_speed([10, 10], 800, 60, 1) 

[0.17453293 0.17453293] 4.29 4.29


In [23]:
# import multiprocessing as mp
# print("Number of processors: ", mp.cpu_count())

In [49]:
# from screeninfo import get_monitors

# def mon_rez():
#     rez = []
#     for monitor in get_monitors():
#         width = monitor.width
#         height = monitor.height
#         rez.append([width, height])
#     return rez
# mon_rez()

In [8]:
import subprocess

# r = 1
def mon_size():
    screens = [l.split() for l in subprocess.check_output(["xrandr"]).decode("utf-8").strip().splitlines() if " connected" in l]

    scr_data = []
    for s in screens:
        if s[2] == 'primary':
            c1 = 'x'
            c2 = '+'
            try:
                strValue = s[-12]
                scr_data.append((
                    float(s[-3].replace("mm", "")),
                    float(s[-1].replace("mm", "")),
                    int(strValue.split(c1,1)[0]),
                    int(strValue.split(c1,1)[1].split(c2,1)[0])                
                    ))
                # print(s[-12])
            except ValueError:
                pass
    dl = []
    dml= []
    
    for s in scr_data:
        wl = s[0]; hl = s[1];
        wp = s[2]; hp = s[3];
        dml.append([round(wl,1), round(hl,1)])
        dml.append([wp, hp])
        # dl.append(round(d,0))
    return dml
mon_size()


[[527.0, 296.0], [1920, 1080]]

In [9]:
import numpy as np
def calc_speed(spd, dfs, fps):
    speed = np.array(spd) * np.pi / 180 #numpy arrray of speed in visual angle
    tsp = np.tan(speed)
    ss = np.array(mon_size()[0]) #screen size in cms
    sr = np.array(mon_size()[1])
    # print(dml.shape)
    sw_r = round(sr[0]/ss[0], 2)
    sh_r = round(sr[1]/ss[1], 2)
    
    print(speed, sw_r, sh_r)
    
    sx = int(tsp[0] * dfs * sw_r / fps)
    sy = int(tsp[1] * dfs * sh_r / fps)
    asx = np.arctan(sx * fps / (sw_r * dfs)) * 180 / np.pi
    asy = np.arctan(sy * fps / (sh_r * dfs)) * 180 / np.pi

    s_ball = [sx, sy]
    av_ball = [asx, asy]
    
    return s_ball, av_ball
    
calc_speed([10, 10], 800, 60)

[0.17453293 0.17453293] 3.64 3.65


([8, 8], [9.360190801913326, 9.334998463138238])

In [10]:
dims = mon_size()[1]
x = max(dims)
div = []
for i in range(1, x+1):
    if x%i == 0:
        div.append(i)
print(div)
y = min(dims)
cmd = []
for j in div:
    if y%j == 0:
        cmd.append(j)
print(cmd)

[1, 2, 3, 4, 5, 6, 8, 10, 12, 15, 16, 20, 24, 30, 32, 40, 48, 60, 64, 80, 96, 120, 128, 160, 192, 240, 320, 384, 480, 640, 960, 1920]
[1, 2, 3, 4, 5, 6, 8, 10, 12, 15, 20, 24, 30, 40, 60, 120]


In [6]:
import math

In [8]:
j = mon_size()[1]
math.gcd(j[0], j[1])

120

In [9]:
import math

# get screen resolution
x_res = int(input("Enter screen resolution (x): "))
y_res = int(input("Enter screen resolution (y): "))

# calculate side length of each quadrant
quad_side = math.sqrt((x_res * y_res) / 16)

# round to nearest integer
quad_side = round(quad_side)

# calculate actual size of cut screen
# cut_x = quad_side * 4
# cut_y = quad_side * 4

print("The biggest square side (in pixels) that can approximately cut the screen into 16 equal sides is:", quad_side)
print("The cut screen size would be:", cut_x, "x", cut_y)

Enter screen resolution (x):  1920
Enter screen resolution (y):  1080


The biggest square side (in pixels) that can approximately cut the screen into 16 equal sides is: 360
The cut screen size would be: 1440 x 1440


In [11]:
import math

def find_square_side(x_res, y_res, square_size):
    n_squares_x = math.ceil(x_res / square_size)
    n_squares_y = math.ceil(y_res / square_size)
    n_squares = n_squares_x * n_squares_y
    total_pixels = x_res * y_res
    square_pixels = total_pixels / n_squares
    square_side = int(math.sqrt(square_pixels))
    return square_side

find_square_side(1920, 1080, 4)

4

In [3]:
from screeninfo import get_monitors
for m in get_monitors():
    print(m)


Monitor(x=0, y=0, width=1920, height=1080, width_mm=527, height_mm=296, name='DP-2', is_primary=True)
Monitor(x=1920, y=0, width=2560, height=1440, width_mm=597, height_mm=336, name='DP-4', is_primary=False)
